In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "las-vegas_nevada.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"
RELATION_TAGS_PATH = "relation_tags.csv"
RELATION_PATH = "relations.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
postcode_NV_re = re.compile(r'[A-Z]+')
postcode_re = re.compile(r'-')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
RELATION_TAGS_FIELDS = ['id', 'key', 'value', 'type']
RELATION_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']

def update_name(name):
    m = re.search(postcode_NV_re, name)
    n = re.search(postcode_re, name)
    if m:
        name = (re.sub(postcode_NV_re, '', name)).strip()
    elif n:
        name = name.split('-')[0]
    return name

def shape_tag(element):
    tags = []
    for child in element:
        if child.tag == 'tag':
            tags_attrib = {}
            tags_attrib['id'] = element.attrib['id']
            problem = re.search(PROBLEMCHARS, child.attrib['k'])
            colon = re.match(LOWER_COLON, child.attrib['k'])
            if problem:
                continue
            else:
                if colon:
                    k_value = child.attrib['k'].split(':', 1)
                    tags_attrib['key'] = k_value[1]
                    if child.attrib['k'] == 'addr:postcode':
                        tags_attrib['value'] = update_name(child.attrib['v'])
                    else:
                        tags_attrib['value'] = child.attrib['v']
                    tags_attrib['type'] = k_value[0]
                    tags.append(tags_attrib)
                else:
                    tags_attrib['key'] = child.attrib['k']
                    tags_attrib['value'] = child.attrib['v']
                    tags_attrib['type'] = 'regular'
                    tags.append(tags_attrib)
    return tags

def shape_way_nodes(element):
    way_nodes = []
    counter = 0
    for ch in element:
        if ch.tag == 'nd':
            way_nodes_attrib = {}
            way_nodes_attrib['id'] = element.attrib['id']
            way_nodes_attrib['node_id'] = ch.attrib['ref']
            way_nodes_attrib['position'] = counter
            counter += 1
            way_nodes.append(way_nodes_attrib)
    return way_nodes


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  re_attr_fields=RELATION_FIELDS,problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    relation_attribs = {}
     # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs = {i: element.attrib[i] for i in node_attr_fields}
        tags = shape_tag(element)
        return {'node': node_attribs, 'node_tags': tags}                
    
    if element.tag == 'way':
        way_attribs = {i_way: element.attrib[i_way] for i_way in way_attr_fields}
        way_nodes = shape_way_nodes(element)
        tags = shape_tag(element)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
    if element.tag == 'relation':
        relation_attribs = {i_re: element.attrib[i_re] for i_re in re_attr_fields}
        tags = shape_tag(element)
        return {'relation': relation_attribs, 'relation_tags': tags}
    


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file, \
         codecs.open(RELATION_PATH, 'w') as relation_file, \
         codecs.open(RELATION_TAGS_PATH, 'w') as relation_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)
        relation_tags_writer = UnicodeDictWriter(relation_tags_file, RELATION_TAGS_FIELDS)
        relation_writer = UnicodeDictWriter(relation_file, RELATION_FIELDS)
        
        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        relation_writer.writeheader()
        relation_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way', 'relation')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                elif element.tag == 'relation':
                    relation_writer.writerow(el['relation'])
                    relation_tags_writer.writerows(el['relation_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)
